In [0]:
import pandas as pd

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import top_k_categorical_accuracy

from sklearn.preprocessing import StandardScaler

In [0]:
df = pd.read_csv('/content/SpotifyTracks_doubleforloop_genre_year.csv',index_col=[0])

In [106]:
df.head()

,artist_name,track_name,track_id,popularity,year,genre,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Gorillaz,On Melancholy Hill,0q6LuUqGLUiCPP1cbdwFs3,75,2010,alternative,0,0.000015,0.689,233867,0.739,0.509000,2,0.0640,-5.810,1,0.0260,120.423,4,0.578
1,Avenged Sevenfold,Nightmare,4UEo1b0wWrtHMC8bVqPiH8,70,2010,alternative,1,0.000318,0.554,374453,0.949,0.000100,2,0.2000,-4.928,1,0.0787,129.984,4,0.233
2,The Black Keys,Howlin' for You,0grFc6klR3hxoHLcgCYsF4,66,2010,alternative,2,0.028000,0.705,191800,0.735,0.078300,11,0.1120,-6.646,1,0.0931,132.627,4,0.448
3,My Darkest Days,Porn Star Dancing,3Q8zopc4ABXhysDb1sgLVW,65,2010,alternative,3,0.013900,0.477,199013,0.917,0.000000,2,0.0756,-3.399,1,0.0837,160.044,4,0.271
4,Volbeat,A Warrior's Call,0hTiTU0yqthnByyZDD3bcc,62,2010,alternative,4,0.000750,0.374,263080,0.903,0.000006,10,0.2440,-4.490,1,0.0825,109.118,3,0.429


In [0]:
genreframe = pd.concat([df,pd.get_dummies(df['genre'], prefix='genre')],axis=1)

In [108]:
genreframe.head()

,artist_name,track_name,track_id,popularity,year,genre,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,genre_alternative,genre_country,genre_dance,genre_folk,genre_grunge,genre_indie,genre_jazz,genre_metal,genre_pop,genre_punk,genre_rap,genre_rock
0,Gorillaz,On Melancholy Hill,0q6LuUqGLUiCPP1cbdwFs3,75,2010,alternative,0,0.000015,0.689,233867,0.739,0.509000,2,0.0640,-5.810,1,0.0260,120.423,4,0.578,1,0,0,0,0,0,0,0,0,0,0,0
1,Avenged Sevenfold,Nightmare,4UEo1b0wWrtHMC8bVqPiH8,70,2010,alternative,1,0.000318,0.554,374453,0.949,0.000100,2,0.2000,-4.928,1,0.0787,129.984,4,0.233,1,0,0,0,0,0,0,0,0,0,0,0
2,The Black Keys,Howlin' for You,0grFc6klR3hxoHLcgCYsF4,66,2010,alternative,2,0.028000,0.705,191800,0.735,0.078300,11,0.1120,-6.646,1,0.0931,132.627,4,0.448,1,0,0,0,0,0,0,0,0,0,0,0
3,My Darkest Days,Porn Star Dancing,3Q8zopc4ABXhysDb1sgLVW,65,2010,alternative,3,0.013900,0.477,199013,0.917,0.000000,2,0.0756,-3.399,1,0.0837,160.044,4,0.271,1,0,0,0,0,0,0,0,0,0,0,0
4,Volbeat,A Warrior's Call,0hTiTU0yqthnByyZDD3bcc,62,2010,alternative,4,0.000750,0.374,263080,0.903,0.000006,10,0.2440,-4.490,1,0.0825,109.118,3,0.429,1,0,0,0,0,0,0,0,0,0,0,0


In [109]:
genreframe.iloc[0,7:20]

acousticness        1.51e-05
danceability           0.689
duration_ms           233867
energy                 0.739
instrumentalness       0.509
key                        2
liveness               0.064
loudness               -5.81
mode                       1
speechiness            0.026
tempo                120.423
time_signature             4
valence                0.578
Name: 0, dtype: object

In [110]:
X_train = genreframe.iloc[:,7:20].to_numpy()
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_scaled

array([[-0.81353437,  0.86273319,  0.01422737, ..., -0.07457545,
         0.21919423,  0.52020022],
       [-0.81253793,  0.02764277,  1.61395879, ...,  0.24680209,
         0.21919423, -0.95936143],
       [-0.72147348,  0.96170687, -0.46445403, ...,  0.33564226,
         0.21919423, -0.03731576],
       ...,
       [ 0.35424623,  1.02975127,  0.17080273, ..., -0.0196848 ,
         0.21919423,  1.42080297],
       [-0.80624809, -0.62187201, -1.57989421, ..., -1.07440547,
         0.21919423,  1.22781667],
       [-0.42211418,  1.23388449, -0.24267685, ..., -0.99696024,
         0.21919423, -0.50048289]])

In [111]:
genres = genreframe.genre.unique().tolist()
genres

['alternative',
 'country',
 'dance',
 'folk',
 'grunge',
 'indie',
 'jazz',
 'metal',
 'pop',
 'punk',
 'rap',
 'rock']

In [112]:
onehots = genreframe.iloc[:,20:]
y_train = onehots.to_numpy()
y_train.shape

(164449, 12)

In [113]:
y_train.shape

(164449, 12)

In [114]:
model = Sequential()

# Input => Hidden
model.add(Dense(26, input_dim=13, activation='relu'))
# Hidden
model.add(Dropout(0.2))
# Hidden
model.add(Dense(26, activation='relu'))
# Hidden
model.add(Dropout(0.2))
# Hidden
model.add(Dense(26, activation='relu'))
# Hidden
model.add(Dropout(0.2))
# Output
model.add(Dense(12,activation='softmax'))

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

#Compile
model.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_53 (Dense)             (None, 26)                364       
_________________________________________________________________
dropout_42 (Dropout)         (None, 26)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 26)                702       
_________________________________________________________________
dropout_43 (Dropout)         (None, 26)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 26)                702       
_________________________________________________________________
dropout_44 (Dropout)         (None, 26)                0         
_________________________________________________________________
dense_56 (Dense)             (None, 12)              

In [115]:
# Fit the model
history = model.fit(X_scaled, y_train, validation_split=0.2, epochs=50, batch_size=1000)

Train on 131559 samples, validate on 32890 samples
Epoch 1/50
131559/131559 [==============================] - 1s 8us/sample - loss: 2.3608 - acc: 0.1520 - val_loss: 2.1331 - val_acc: 0.2463
Epoch 2/50
131559/131559 [==============================] - 1s 4us/sample - loss: 2.1149 - acc: 0.2614 - val_loss: 1.9700 - val_acc: 0.3562
Epoch 3/50
131559/131559 [==============================] - 1s 4us/sample - loss: 2.0443 - acc: 0.2930 - val_loss: 1.9370 - val_acc: 0.3648
Epoch 4/50
131559/131559 [==============================] - 1s 4us/sample - loss: 2.0169 - acc: 0.3048 - val_loss: 1.9232 - val_acc: 0.3676
Epoch 5/50
131559/131559 [==============================] - 1s 4us/sample - loss: 2.0009 - acc: 0.3097 - val_loss: 1.9151 - val_acc: 0.3714
Epoch 6/50
131559/131559 [==============================] - 1s 4us/sample - loss: 1.9895 - acc: 0.3146 - val_loss: 1.9086 - val_acc: 0.3724
Epoch 7/50
131559/131559 [==============================] - 1s 4us/sample - loss: 1.9819 - acc: 0.3176 - val_

In [118]:
obs = genreframe.iloc[10000:10001,7:20].to_numpy()
obs.shape

(1, 13)

In [119]:
model.predict(obs)

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

In [121]:
genreframe.iloc[10000:10001,5]

10000    jazz
Name: genre, dtype: object

In [0]:
model.save('genre_NN')